In [94]:
%matplotlib notebook
import lxml.html as lh
import time
import urllib.request
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
import csv


Traceback (most recent call last):
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\cbook\__init__.py", line 215, in process
    func(*args, **kwargs)
  File "C:\Users\Admin\AppData\Local\Programs\Python\Python36\lib\site-packages\matplotlib\animation.py", line 1478, in _stop
    self.event_source.remove_callback(self._loop_delay)
AttributeError: 'NoneType' object has no attribute 'remove_callback'
2020-05-22 13:58:22 DESKTOP-F7HNRJU matplotlib.pyplot[18404] DEBUG Loaded backend nbAgg version unknown.


In [95]:
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

In [96]:
tz = pytz.timezone('Asia/Kolkata')

In [97]:
path1 = os.getcwd()
path = path1 + '/chromedriver'

In [98]:
ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
driver = webdriver.Chrome(executable_path=path , options=options)

2020-05-22 13:58:26 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG POST http://127.0.0.1:63660/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}, "desiredCapabilities": {"browserName": "chrome", "version": "", "platform": "ANY", "goog:chromeOptions": {"excludeSwitches": ["enable-logging"], "extensions": [], "args": ["--no-sandbox", "--disable-dev-shm-usage", "--headless"]}}}
2020-05-22 13:58:26 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG Starting new HTTP connection (1): 127.0.0.1:63660
2020-05-22 13:58:27 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "POST /session HTTP/1.1" 200 717
2020-05-22 13:58:27 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request


In [99]:
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)
    else:
        driver.refresh()
        time.sleep(15)
    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

In [109]:
def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args + '.NS'
    price = []
    window_size =10
    
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(9, 14, tzinfo=tz) < time_now < datetime.time(15, 31, tzinfo=tz)):
        url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(ticker,ticker)
        print(count)
        live = Real(url,count)
        current_time = time.ctime()[11:19]
        live = Real(url,count)
        # appending the data in csv
        with open('data.csv','a') as f:
            csv_writer = csv.DictWriter(f,fieldnames=['time','price'])
            info = {
                'time':current_time,
                'price':live
            }
            csv_writer.writerow(info)
            print(current_time,live)
            live = Real(url,count)
            current_time = time.ctime()[11:19]
        time.sleep(0.1)
    
            
        count+=1        
        price.append(live)
        if count < window_size:
            continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        profit = evaluate_model(agent,state,next_state, price, t, total_profit, history, reward, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])
        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta
        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))
    agent.memory.append((state, action, reward, next_state))
    return total_profit

In [112]:
# input is the ticker to pass to the main function 
#instead of argparse I have used input 
args = input()
    
coloredlogs.install(level="DEBUG")
switch_k_backend_device()
with open('data.csv','a') as f:
    csv_writer = csv.DictWriter(f,fieldnames=['time','price'])
    csv_writer.writeheader()
try:
    main(args)
except KeyboardInterrupt:
    print("Aborted")
    #Destroying the csv file
    os.remove('data.csv')


RELIANCE


2020-05-22 15:03:05 DESKTOP-F7HNRJU root[18404] DEBUG switching to TensorFlow for CPU


0


2020-05-22 15:03:05 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG POST http://127.0.0.1:63660/session/4bcd77050c76677fd921469331b32d1b/url {"url": "https://finance.yahoo.com/quote/RELIANCE.NS?p=RELIANCE.NS&.tsrc=fin-srch"}
2020-05-22 15:03:08 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "POST /session/4bcd77050c76677fd921469331b32d1b/url HTTP/1.1" 200 14
2020-05-22 15:03:08 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request
2020-05-22 15:03:08 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG GET http://127.0.0.1:63660/session/4bcd77050c76677fd921469331b32d1b/source {}
2020-05-22 15:03:08 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "GET /session/4bcd77050c76677fd921469331b32d1b/source HTTP/1.1" 200 871174
2020-05-22 15:03:09 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request
2020-05-22 15:03:09 DESKT

15:03:09 1434.0


2020-05-22 15:03:11 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG POST http://127.0.0.1:63660/session/4bcd77050c76677fd921469331b32d1b/url {"url": "https://finance.yahoo.com/quote/RELIANCE.NS?p=RELIANCE.NS&.tsrc=fin-srch"}
2020-05-22 15:03:15 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "POST /session/4bcd77050c76677fd921469331b32d1b/url HTTP/1.1" 200 14
2020-05-22 15:03:15 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request
2020-05-22 15:03:15 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG GET http://127.0.0.1:63660/session/4bcd77050c76677fd921469331b32d1b/source {}
2020-05-22 15:03:16 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "GET /session/4bcd77050c76677fd921469331b32d1b/source HTTP/1.1" 200 868880
2020-05-22 15:03:16 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request


1


2020-05-22 15:03:16 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG POST http://127.0.0.1:63660/session/4bcd77050c76677fd921469331b32d1b/refresh {}
2020-05-22 15:03:20 DESKTOP-F7HNRJU urllib3.connectionpool[18404] DEBUG http://127.0.0.1:63660 "POST /session/4bcd77050c76677fd921469331b32d1b/refresh HTTP/1.1" 200 14
2020-05-22 15:03:20 DESKTOP-F7HNRJU selenium.webdriver.remote.remote_connection[18404] DEBUG Finished Request


Aborted


# Visualization

In [113]:
def graph(i):
    with open('data.csv','a') as f:
        csv_writer = csv.DictWriter(f,fieldnames=['time','price'])
        csv_writer.writeheader()
    df = pd.read_csv('data.csv')
    x=df['time']
    y=df['price']

    plt.clf()
    plt.plot(x,y,label = '1')
g = FuncAnimation(plt.gcf(),graph,interval = 1000)
plt.tight_layout()
plt.show()